In [ ]:
import sqlite3
import pandas as pd
import seaborn as sns
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [113]:
# set up db connection
conn = sqlite3.connect("data/challenge_db.db")

In [114]:
# get data frame for plots
rank_two = pd.read_sql("""
WITH incl_rank AS (SELECT SUM(totalAmount) as total, AVG(totalAmount) as average, MAX(totalAmount), locationId, city,
RANK() OVER (
PARTITION BY city
ORDER BY city ASC, MAX(totalAmount) DESC
) AS order_rank

FROM payments JOIN locations ON payments.locationId=locations.uuid
WHERE NOT payments.status = "ERR"
GROUP BY city, locationId
ORDER BY city ASC, MAX(totalAmount) DESC)

SELECT *
FROM incl_rank
WHERE order_rank = 2
""", con = conn)

In [119]:
def plot_spec_city(city):
    plt = sns.barplot(y = pd.Series([rank_two.loc[rank_two.city == city,"average"].values[0], 
                                     rank_two.loc[rank_two.city == city,"total"].values[0]], 
                              index= [0, 1]), 
                x = pd.Series(["average", "total"]))
    plt.bar_label(plt.containers[0])
    

In [120]:
interact(plot_spec_city, city = rank_two.city.values)

interactive(children=(Dropdown(description='city', options=('Berlin', 'Hamburg', 'Ot'), value='Berlin'), Outpu…

<function __main__.plot_spec_city(city)>